In [ ]:
import jtrace
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline

In [ ]:
D = 0.1
m0 = jtrace.ConstMedium(1.000277)  # air
w, n = np.genfromtxt(os.path.join(jtrace.datadir, "media", "silica_dispersion.txt")).T
w *= 1e-6
m1 = jtrace.TableMedium(jtrace.Table(w, n, jtrace.Table.Interpolant.linear))
wavelength = 500e-9
n0 = m0.getN(wavelength)
n1 = m1.getN(wavelength)
R1 = 0.5  # convex entrance surface
R2 = -0.5  # convex exit surface
d = 0.01
# Lens maker equation:
f = 1./((n1-n0)*(1./R1 - 1./R2 + (n1-n0)*d/R1/R2))

In [ ]:
s1 = jtrace.Quadric(R1, 0, -d/2)
s2 = jtrace.Quadric(R2, 0, d/2)

In [ ]:
x = np.linspace(-0.05, 0.05, 100)
plt.plot(x, [s1.sag(0, x_) for x_ in x])
plt.plot(x, [s2.sag(0, x_) for x_ in x])
plt.axes().set_aspect('equal', 'datalim')

In [ ]:
z_origin = -0.02
def rays(theta_x, theta_y, wavelength):
    # Point towards (0,0,0), but at an angle.  Need to determine pupil locations.
    rs = np.linspace(0, D/2, 10)
    # The above works if theta is 0.
    # If theta is not zero, then need to shift the rays depending on how far away their origins are.
    # We'll set the z-origin to be in the same plane as the focal plane.
    dx = f * np.tan(theta_x)
    dy = f * np.tan(theta_y)
    rays_ = []
    for r in rs:        
        phis = np.linspace(0, 2*np.pi, int(np.ceil(r/D*64)), endpoint=False)
        for phi in phis:
            rays_.append(
                jtrace.Ray(jtrace.Vec3(r*np.cos(phi)+dx, r*np.sin(phi)+dy, z_origin),
                           jtrace.Vec3(-np.tan(theta_x), -np.tan(theta_y), +1).UnitVec3()/1.000277,
                           0, wavelength))
    return rays_

In [ ]:
@interact(theta_x=widgets.FloatSlider(min=-1,max=1,step=0.01,value=0.1),
          theta_y=widgets.FloatSlider(min=-1,max=1,step=0.01,value=0.1),
          wavelength=widgets.FloatSlider(min=0.4,max=0.7,step=0.01,value=0.5),
          focus=widgets.FloatSlider(min=-20, max=20, step=0.1,value=5))
def spot(theta_x, theta_y, wavelength, focus):
    """Display a spot diagram for a Newtonian telescope.

    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param focus    Defocus distance in mm
    """
    focal_plane = jtrace.Plane(f+focus*1e-3)
    spots = []
    for ray in rays(theta_x*np.pi/180, theta_y*np.pi/180, wavelength*1e-6):
        isec1 = s1.intersect(ray)
        rray1 = isec1.refractedRay(ray, m0, m1)

        isec2 = s2.intersect(rray1)
        rray2 = isec2.refractedRay(rray1, m1, m0)

        isec3 = focal_plane.intersect(rray2)
        spots.append([isec3.x0, isec3.y0])
    spots = np.array(spots)
    spots -= np.mean(spots, axis=0)
#     spots *= plate_scale*206265 # meters -> arcseconds
    plt.figure(figsize=(4.5,4))
    plt.scatter(spots[:,0], spots[:,1], s=1, alpha=0.5)
#     plt.xlim(-10, 10)
#     plt.ylim(-10, 10)
    plt.title(r"$\theta_x = {:4.2f}\,,\theta_y = {:4.2f}\,,\lambda = {:4g}\,, f={:4.2f}$".format(theta_x, theta_y, wavelength, focus))
    plt.xlabel("meters")
    plt.ylabel("meters")
    plt.axes().set_aspect('equal', 'datalim')